This notebook implements the code from [backtrader tutorial](https://www.backtrader.com/docu/q.uickstart/quickstart/)

In [4]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

In [7]:
import os
import sys
import yfinance as yf
import backtrader as bt
from datetime import datetime

import pandas as pd

In [5]:
# Add project folder to system path.
PROJECT_FOLDER = "/Users/wtai/Projects/Quantitative_trading/"
sys.path.append(PROJECT_FOLDER)
import src.utils as utils

In [3]:
# Download data
# tenb_df = utils.download_yf("TENB", "2y", "1d", "/Users/wtai/Projects/Quantitative_trading/data")
tenb_df = pd.read_csv("/Users/wtai/Projects/Quantitative_trading/data/TENB_20210527_20230526_1d.csv")
tenb_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2021-05-27 00:00:00-04:00,42.160000,42.299999,41.130001,42.049999,645700,0.0,0.0
1,2021-05-28 00:00:00-04:00,42.439999,42.540001,41.709999,41.799999,357900,0.0,0.0
2,2021-06-01 00:00:00-04:00,41.820000,41.990002,40.590000,41.230000,1022600,0.0,0.0
3,2021-06-02 00:00:00-04:00,41.169998,41.470001,40.660000,40.990002,570900,0.0,0.0
4,2021-06-03 00:00:00-04:00,40.630001,40.700001,39.779999,40.380001,516100,0.0,0.0
...,...,...,...,...,...,...,...,...
499,2023-05-22 00:00:00-04:00,39.790001,40.720001,39.240002,40.200001,1253100,0.0,0.0
500,2023-05-23 00:00:00-04:00,39.860001,40.220001,37.380001,37.400002,1189100,0.0,0.0
501,2023-05-24 00:00:00-04:00,36.959999,38.779999,36.540001,38.450001,1006500,0.0,0.0
502,2023-05-25 00:00:00-04:00,38.720001,38.939999,37.360001,37.630001,766800,0.0,0.0


In [16]:
# First strategy
class TestStrategy(bt.Strategy):
    """Test strategy."""
    def log(self, txt:str, dt:datetime=None) -> None:
        """Print text to console."""
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))
    
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] data series.
        self.dataclose = self.datas[0].close
    
    def next(self):
        """Moving to the next day."""
        self.log("Close, %.2f" % self.dataclose[0])

Use backtrader to backtest a simple strategy.

In [19]:
# Data file
data_file = "TENB_20210527_20230526_1d.csv"
# Create Cerebro engine.
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(TestStrategy)
# Crate a data object from local CSV data downloaded from YahooFinance.
# The YahooFinanceCSVData does not comply with today's YahooFinance data.
# Hence use GenericCSVData instead.
data = bt.feeds.GenericCSVData(
    dataname=os.path.join(PROJECT_FOLDER, "data", data_file),
    # Datetime format
    dtformat='%Y-%m-%d %H:%M:%S%z',
    # Do not pass values before this date.
    fromdate=datetime(2021, 5, 27),
    # Do not pass values after this date.
    todate=datetime(2023, 5, 26),
    # Datetime column position
    datetime=0,
    # Time column not exist (-1)
    time=-1,
    # Open column position
    open=1,
    # High column position
    high=2,
    # Low column position
    low=3,
    # Close column position
    close=4,
    # Volume column position
    volume=5,
    # Reverse ordered
    reverse=False
)
# Add data object to the Cerebro engine.
cerebro.adddata(data)
# Set the initial cash value.
cerebro.broker.setcash(1e5)
# Run backtesting
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2021-05-27, Close, 42.05
2021-05-28, Close, 41.80
2021-06-01, Close, 41.23
2021-06-02, Close, 40.99
2021-06-03, Close, 40.38
2021-06-04, Close, 41.78
2021-06-07, Close, 42.60
2021-06-08, Close, 42.96
2021-06-09, Close, 42.27
2021-06-10, Close, 42.99
2021-06-11, Close, 43.47
2021-06-14, Close, 43.88
2021-06-15, Close, 42.90
2021-06-16, Close, 42.90
2021-06-17, Close, 43.32
2021-06-18, Close, 43.80
2021-06-21, Close, 43.57
2021-06-22, Close, 45.02
2021-06-23, Close, 44.36
2021-06-24, Close, 43.50
2021-06-25, Close, 43.69
2021-06-28, Close, 42.93
2021-06-29, Close, 42.04
2021-06-30, Close, 41.35
2021-07-01, Close, 40.70
2021-07-02, Close, 40.56
2021-07-06, Close, 42.71
2021-07-07, Close, 42.46
2021-07-08, Close, 41.93
2021-07-09, Close, 41.94
2021-07-12, Close, 41.20
2021-07-13, Close, 40.89
2021-07-14, Close, 39.74
2021-07-15, Close, 39.65
2021-07-16, Close, 39.50
2021-07-19, Close, 39.83
2021-07-20, Close, 42.11
2021-07-21, Close, 43.12
2021-07-22, Cl